# NOTES: Kaggle - Intro to Machine Learning

Neste notebook está presente algumas anotação sobre o curso de [intro to Machine Learning](https://www.kaggle.com/learn/intro-to-machine-learning) do [Kaggle](https://www.kaggle.com/)

In [12]:
import pandas as pd
from IPython.display import display

## Sobre os Dataset

In [3]:
iowa_file_path = '../datas/kaggle:home_data_for_ml_course.csv'
home_data = pd.read_csv(iowa_file_path)

y = home_data.SalePrice

feature_columns = ['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd']
X = home_data[feature_columns]

In [13]:
display(X.describe())

display(X.head())

,LotArea,YearBuilt,1stFlrSF,2ndFlrSF,FullBath,BedroomAbvGr,TotRmsAbvGrd
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,10516.828082,1971.267808,1162.626712,346.992466,1.565068,2.866438,6.517808
std,9981.264932,30.202904,386.587738,436.528436,0.550916,0.815778,1.625393
min,1300.000000,1872.000000,334.000000,0.000000,0.000000,0.000000,2.000000
25%,7553.500000,1954.000000,882.000000,0.000000,1.000000,2.000000,5.000000
50%,9478.500000,1973.000000,1087.000000,0.000000,2.000000,3.000000,6.000000
75%,11601.500000,2000.000000,1391.250000,728.000000,2.000000,3.000000,7.000000
max,215245.000000,2010.000000,4692.000000,2065.000000,3.000000,8.000000,14.000000


,LotArea,YearBuilt,1stFlrSF,2ndFlrSF,FullBath,BedroomAbvGr,TotRmsAbvGrd
0,8450,2003,856,854,2,3,8
1,9600,1976,1262,0,2,3,6
2,11250,2001,920,866,2,3,6
3,9550,1915,961,756,1,3,7
4,14260,2000,1145,1053,2,4,9


## Modelo de Predição: Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [5]:
iowa_model = DecisionTreeRegressor(random_state=1)

iowa_model.fit(X, y)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=1, splitter='best')

In [18]:
print("First in-sample predictions:", iowa_model.predict(X.head()))
print("Actual target values for those homes:", y.head().tolist())

First in-sample predictions: [208500. 181500. 223500. 128000. 250000.]
Actual target values for those homes: [208500, 181500, 223500, 140000, 250000]


## Treino

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
train_X, val_X, train_y, val_y = train_test_split(X,y,random_state=1)

In [40]:
iowa_model = DecisionTreeRegressor(random_state=1)

iowa_model.fit(train_X,train_y)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=1, splitter='best')

In [9]:
val_predictions = iowa_model.predict(val_X)

In [39]:
print(val_predictions[:10])
print(val_y[:10])

[182392.31944444 163142.5        125362.57333333  70167.58333333
 146415.0075188  332156.         270325.         159006.60714286
 228735.80645161 268500.        ]
258     231500
267     179500
288     122000
649      84500
1233    142000
167     325624
926     285000
831     151000
1237    195000
426     275000
Name: SalePrice, dtype: int64


## Validação do Modelo

In [30]:
from sklearn.metrics import mean_absolute_error
val_mae = mean_absolute_error(val_y,val_predictions)
print(val_mae)

29652.931506849316


## Aprimorando o Modelo

In [23]:
def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

In [25]:
candidate_max_leaf_nodes = [5, 25, 50, 100, 250, 500]

scores = {}
for leaf in candidate_max_leaf_nodes:
    scores[leaf] = get_mae(leaf, train_X, val_X, train_y, val_y)
    
best_tree_size = min(scores, key=scores.get)
print(best_tree_size)

100


In [37]:
final_model = DecisionTreeRegressor(max_leaf_nodes=best_tree_size)

final_model.fit(X, y)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=100, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

In [38]:
val_predictions = final_model.predict(val_X)
val_mae = mean_absolute_error(val_y,val_predictions)

print(val_mae)

16815.938748057826


## Modelo de Predição: Random Forests

In [28]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

forest_model = RandomForestRegressor(n_estimators=100,random_state=1)
forest_model.fit(train_X, train_y)
melb_preds = forest_model.predict(val_X)
print(mean_absolute_error(val_y, melb_preds))

21857.15912981083
